In [12]:
import pandas as pd
import altair as alt
import sys


In [13]:
def create_df(highest_rank, compare, successTable):
    x = range(1,highest_rank)
    denom = len(successTable)
    data=[]
    for runtype in compare:
        success = successTable.loc[(successTable['Variant_Level_noMOI_'+str(runtype)]=='Variant_Present_noMOI') | (successTable['Variant_Level_noMOI_'+str(runtype)]=='Variant_Present_noMOI')]
        for i in x:
            num = len(success.loc[success['Variant_Level_noMOI_rank_'+str(runtype)] <= i])
            data.append([i,num, (num/denom)*100, runtype])
        print('highest rank:', max(success['Variant_Level_noMOI_rank_'+str(runtype)]), 'max%:', (num/denom)*100, num)

    df = pd.DataFrame(data, columns=['Rank', 'NumPatients', 'Percent_Variants', 'Run_Type'])
    return df, denom

In [14]:
def create_plot(denom, source, compare, color_scheme, domain):
    title=str(denom) +'Diagnostic Variants (Variant Level - no MOI requirement)'
    bigChart = alt.Chart(source, title=title).mark_line().encode(
        x=alt.X('Rank', title='Exomiser Rank of Causal Variant'),
        y=alt.Y('Percent_Variants', title='Percent of Causal Variants within Exomiser Rank', scale=alt.Scale(domain=[0,100])),
        color=alt.Color('Run_Type:N',legend=None,sort=compare,scale=alt.Scale(domain=domain,scheme=color_scheme)),
        tooltip=['Rank', 'Percent_Variants', 'Run_Type']
    ).properties(
        width=600,
        height=500)
    
    zoom_source = source.loc[source['Rank'] <=30]

    zoomChart = alt.Chart(zoom_source, title=title).mark_line(point=alt.OverlayMarkDef(size=50)).encode(
        x=alt.X('Rank', title='Exomiser Rank of Causal Variant'),
        y=alt.Y('Percent_Variants', title='Percent of Causal Variants within Exomiser Rank', scale=alt.Scale(domain=[0,100])),
        color=alt.Color('Run_Type:N', sort=domain, scale=alt.Scale(domain=domain,scheme=color_scheme)),
        tooltip=['Rank', 'Percent_Variants', 'Run_Type']
    ).properties(
        width=400,
        height=300)
    plot=alt.vconcat(bigChart, zoomChart)#.configure_legend(labelLimit=0).configure_axis(
        # labelFontSize=15,
        # titleFontSize=15).configure_legend(labelLimit=0,labelFontSize=15, titleFontSize=15)#.configure_axis(grid=False)
    plot=alt.vconcat(zoomChart)
    return plot


In [15]:

success_table = pd.read_csv('supp_fig1_input.txt', sep='\t')
print(len(set(success_table['ID'])),'genes')
print(len(success_table), 'variants')

239 genes
296 variants


In [16]:
domain = ['filtered_exomiser_default', 'filtered_default_genomiser_genomiser']
run_types = ['filtered_exomiser_default', 'filtered_default_genomiser_genomiser']
gs_exomiser_df,denom = create_df(300, run_types, success_table)
gs_exomiser = create_plot(denom, gs_exomiser_df, domain, 'set1', domain)
gs_exomiser

highest rank: 330.0 max%: 95.94594594594594 284
highest rank: 4232.0 max%: 71.95945945945947 213


alt.VConcatChart(...)

In [20]:
run_types = pd.read_csv('supp_fig1_runtimes.tsv', sep='\t')
#run_types = run_types[run_types['cohort']=='GS_Exomiser_Exomiser']
box_plot = alt.Chart(run_types, title='GS_Exomiser_Cohort').mark_boxplot().encode(
        x=alt.X('minutes', title='Run Time (m)'),
        y=alt.Y('cohort', title='Cohort'),
        color=alt.Color('cohort:N', sort=['GS_Exomiser_Exomiser' , 'GS_Exomiser_Genomiser'], scale=alt.Scale(domain=['GS_Exomiser_Exomiser' , 'GS_Exomiser_Genomiser'],scheme='set1')),
    
    ).properties(
        width=400,
        height=100)#.configure_axis(grid=False)

box_plot

alt.Chart(...)

In [18]:

alt.vconcat(gs_exomiser,box_plot).resolve_scale(color='independent').configure_legend(labelLimit=0, labelFontSize=15, titleFontSize=15).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    ).configure_scale(
    bandPaddingInner=0.9
    )

alt.VConcatChart(...)